In [0]:
%sql
-- ============================================================
-- NOTEBOOK C: silver_person_story_status
-- Manual flag table — same pattern as silver_document_person_override
-- Run once to create; then maintain by INSERT/UPDATE as stories are written
-- ============================================================

In [0]:
%sql
-- ── CELL 1: Create table ─────────────────────────────────────────────────────

CREATE TABLE IF NOT EXISTS genealogy.silver_person_story_status (
  person_gedcom_id  STRING  NOT NULL,
  story_written     BOOLEAN NOT NULL,
  story_title       STRING,           -- optional: working title of the story
  story_doc_id      STRING,           -- optional: Google Drive doc ID if saved
  story_url         STRING,           -- optional: URL if published
  published_at      DATE,             -- date published on blog
  notes             STRING,           -- e.g. "Draft complete, needs photos"
  updated_at        TIMESTAMP         -- when this row was last changed
);

In [0]:
%sql
-- ── CELL 2: Example inserts (edit as needed) ─────────────────────────────────
-- Add a row when a story is completed.
-- story_written = TRUE suppresses the person from narrative priority queue
-- via the SIGNAL_STORY_WRITTEN negative weight (-50 in ref_signal_weights).

TRUNCATE TABLE genealogy.silver_person_story_status;

INSERT INTO genealogy.silver_person_story_status VALUES
   ('@I_967018133@', TRUE,  'A credit to his country', '1LEQQUlaAyeFaN2vQ5NHouCz9YYK5fXvAQuryRZqx0_4', 'https://wearetheballs.uk/2022/01/06/a-credit-to-his-country/', '2022-01-06', NULL, current_timestamp())
--   ('@I042@', FALSE, 'Ball family — draft in progress', NULL, NULL, 'Awaiting 1881 census image', current_timestamp());

In [0]:
%sql
-- ── CELL 3: Verification query ───────────────────────────────────────────────
-- Run after inserts to confirm the signal is firing correctly.

SELECT
  s.person_gedcom_id,
  p.given_name,
  p.surname,
  s.story_written,
  s.story_title,
  s.notes
FROM genealogy.silver_person_story_status s
JOIN genealogy.gold_person_life p ON p.person_gedcom_id = s.person_gedcom_id
ORDER BY s.story_written DESC, p.surname, p.given_name;